In [ ]:
# todo:
# Fill nan value
# check feature cols

In [14]:
# models to make today:
# model using only gender
# model using gender + class
# model using gender + class + age (binning)
# model using gender + class + age + name (synthetic feature)  (optional)
# make sure to attach tensorboard and screenshot training curve
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn import metrics
from tensorflow.python.data import Dataset
from sklearn.model_selection import train_test_split

from IPython import display

# for sex_to_int() function
from sklearn.preprocessing import LabelEncoder

# make TensorFlow less verbose
tf.logging.set_verbosity(tf.logging.ERROR)

train_path = '../input/train.csv'
test_path = '../input/test.csv'

In [41]:
class get_data:
    def __init__(self, path, is_train=True):
        self.is_train = is_train
        
        self.feature = pd.read_csv(path)
        self.get_feature_label()
    
    def feature_eng(self):
        self.fill_nan()
        self.drop_cols()
        self.sex_to_int()
    
    def get_feature_label(self):
        if self.is_train:
            self.label = self.feature['Survived']
            self.feature = self.feature.drop('Survived', axis=1)
    
    def fill_nan(self):
        self.df.fillna(0, inplace=True)
    
    def drop_cols(self):
        UNUSED_COLUMNS = ["Ticket", "Cabin", "Embarked", "Fare", "SibSp", "Parch", 'PassengerId']
        self.df = self.df.drop(UNUSED_COLUMNS, axis=1)
        
    def sex_to_int(self):
        le = LabelEncoder()
        le.fit(["male","female"])
        self.df["Sex"]=le.transform(data["Sex"])
        
    def get_dum(self):
        columns = ["Pclass"]
        for column in columns:
            data = pd.concat([data, pd.get_dummies(data[column], prefix=column)], axis=1)
            data = data.drop(column, axis=1)
            
    def split_t_v(self):
        self.ts_f, self_vs_f, self.ts_l, self.vs_l = \
        train_test_split(
        
        )

In [19]:
class create_model:
    def __init__(
        self,
        feature,
        learning_rate=0.0001,
        batch_size=100,
        save_model=True,
        load_model=False,
        save_model_dir='default_model',
        load_model_dir='default_model'
    ):
        self.feature = feature
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        self.save_model = save_model
        self.save_model_dir = save_model_dir
        self.load_model = load_model
        self.load_model_dir = load_model_dir
        
        model_par = dict(
            feature_columns = feature_columns,
            optimizer = self.optimizer,
            model_dir = model_dir
        )
        if self.save_model:
            model_par['model_dir'] = self.save_model_dir
        if self.load_model:
            model_par['warm_start_from'] = self.load_model_dir
        self.model = tf.estimator.LinearRegressor(model_par)
        
    def get_feature_cols(self):
        tmp_feature = [tf.feature_column.numeric_column(my_feature) for my_feature in self.feature]
#         if bin_age: # boolean:
#             tmp_age = tf.feature_column.numeric_column("Age")
#             bucketized_age = tf.feature_column.bucketized_column(
#               tmp_age, boundaries=get_quantile_based_boundaries(
#                 input_features["Age"], 4))
#             tmp_feature += [bucketized_age]
        return set(tmp_feature)
    
    def create_optimizer(self):
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learnint_rate)
        
        '''
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
        self.optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_opt, 2.0)
        '''
    
    def train(
        self,
        ts_f,
        ts_l,
        vs_f,
        vs_l
    ):
        self.ts_f = ts_f
        self.ts_l = ts_l
        self.vs_f = vs_f
        self.vs_l = vs_l
        self.create_input_fn()
        
        steps_per_period = self.steps / self.periods
        for period in range(self.periods):
            self.model.train(
                input_fn=self.train_input_fn,
                steps=steps_per_period
            )
            self.model.evaluate(
                input_fn=self.train_eval_fn
            )
            
            print(" period %02d finish" % (period))
        print("Model training finished.")
        
    def create_input_fn(self):
        self.train_input_fn = lambda: self.input_fn(self.ts_f, self.ts_l, batch_size=self.batch_size)
        self.train_eval_fn = lambda: self.input_fn(self.vs_f, self.vs_l, shuffle=False)
        
    def my_input_fn(self, features, targets, batch_size=1, shuffle=True, num_epochs=None):
        # Convert pandas data into a dict of np arrays.
        features = {key:np.array(value) for key,value in dict(features).items()}
        # Construct a dataset, and configure batching/repeating.
        ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
        ds = ds.batch(batch_size).repeat(num_epochs)
        # Shuffle the data, if specified.
        if shuffle:
            ds = ds.shuffle(42)
        # Return the next batch of data.
        features, labels = ds.make_one_shot_iterator().get_next()
        return features, labels
    
    def predict(self, test_df):
        print('predict')

In [43]:
data = get_data(train_path)
# test = get_data(test_path)
data.feature.head(5)
# data.df.count()
# test.df.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:
model = create_model(save_model=False)

NameError: name 'feature_columns' is not defined